# preprocessing achilles

In [ ]:
from __future__ import print_function
import pandas as pd
import numpy as np

from depmapomics import cn
from genepy import rna
from genepy.utils import helper as h
from genepy.google import gcp
from genepy import mutations as mut
from gsheets import Sheets
from taigapy import TaigaClient


from scipy.stats import pearsonr,spearmanr

from bokeh.plotting import output_notebook
from IPython.display import Image,display
from matplotlib import pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
# do the first steps of https://medium.com/craftsmenltd/from-csv-to-google-sheet-using-python-ef097cb014f9
creds = '../.credentials.json'

sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

refsheet_url = "https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY"


bad = ["ACH-001011",
"ACH-001108",
"ACH-001187",
"ACH-002291" ## added for some reason?
"ACH-002010", # much more than that..
"ACH-000314"
]

lost = set(["ACH-002217",
"ACH-002335",
"ACH-002378"])

## boot up

we are instanciating all the parameters needed for this pipeline to run

In [ ]:
samplesetname = "21Q2"
release=samplesetname

## Do some checks and manual rescuing

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)

In [ ]:
legacy_segments = tc.get(name='depmap-wes-cn-data--08f3', file='legacy_segments')
legacy_segments = legacy_segments.drop(columns='Unnamed: 0')
legacy_segments['Status']='U'
legacy_segments.loc[legacy_segments[legacy_segments.Chromosome.str.contains("chr")].index, "Chromosome"] = [i[3:] for i in legacy_segments[legacy_segments.Chromosome.str.contains("chr")].Chromosome]

prevgenecn = (2**tc.get(name='depmap-a0ab', file='CCLE_gene_cn'))-1 
prevsegments = tc.get(name='depmap-a0ab', file='CCLE_segment_cn')
prev = prevgenecn.index.tolist()

In [ ]:
priosegments = pd.read_csv("temp/segments_allWES_latest_"+samplesetname+".csv")
priogenecn = pd.read_csv('temp/gene_cn_allWES_latest_'+samplesetname+".csv", index_col=0)
priosegments.DepMap_ID.sort_values(), len(set(priosegments.DepMap_ID)), priogenecn.shape

In [ ]:
cyto = pd.read_csv('data/hg38_cytoband.gz', sep='\t',names=['chrom', 'start', 'end','loc','stains'])
cyto['chrom'] = [i[3:] for i in cyto['chrom']]
cyto = cyto.iloc[:-1]
gene_mapping = pd.read_csv('data/genemapping_19Q1.csv')
onlyinleg = set(legacy_segments.DepMap_ID) - set(priosegments.DepMap_ID)
#samegenes = set(prevgenecn.columns) & set(priogenecn.columns)

onlyinleg = onlyinleg - set(bad)
onlyinleg

In [ ]:
for val in onlyinleg:
    print(val,legacy_segments[legacy_segments.DepMap_ID==val].Source.values[0])

## Achilles repriorization

In [ ]:
corr = {}
ge = np.log2(1+priogenecn[samegenes])
pre = np.log2(1+prevgenecn[samegenes]).fillna(0)
for sample in set(prevgenecn.index) & set(priogenecn.index):
        corr[sample] = pearsonr(ge.loc[sample],pre.loc[sample])[0]   
toreplace = []
for k, val in corr.items():
    if val<0.85:
        toreplace.append(k)
len(toreplace)/len(corr)

In [ ]:
a = np.array(list(corr.values()))
sns.kdeplot(a)

In [ ]:
ind = set(prevgenecn.index) & set(priogenecn.index)
sns.scatterplot(x=ge.loc[ind].values.ravel()[:100000],y=pre.loc[ind].values.ravel()[:100000],)

In [ ]:
sns.kdeplot(data=np.array([ge.loc[ind].values.ravel()[:100000], pre.loc[ind].values.ravel()[:100000]]).T, fill=True)

In [ ]:
#mergedsegments = priosegments[~priosegments.DepMap_ID.isin(toreplace)].append(legacy_segments[legacy_segments.DepMap_ID.isin(toreplace)]).reset_index(drop=True)
mergedsegments = priosegments

In [ ]:
mergedsegments = mergedsegments[~mergedsegments.DepMap_ID.isin(list(onlyinleg))].append(legacy_segments[legacy_segments.DepMap_ID.isin(list(onlyinleg))]).reset_index(drop=True)

### Adding WGS

In [ ]:
wgssegments = pd.read_csv("temp/segments_allWGS_latest_"+samplesetname+".csv")
#wgssegments = pd.read_csv("temp/segments_allWGS_latest_"+samplesetname+".csv")
wgssegments.Segment_Mean.max()

In [ ]:
mergedsegments = wgssegments.append(mergedsegments[~mergedsegments.DepMap_ID.isin(set(wgssegments.DepMap_ID))])

In [ ]:
#only in snp
onlyinleg - ( lost | set(wgssegments.DepMap_ID))

### removing normal lines

In [ ]:
normals = ccle_refsamples[ccle_refsamples['primary_disease']=='normal'].index.tolist()
normals

In [ ]:
#mergedsegments = mergedsegments[~mergedsegments.DepMap_ID.isin(normals)]

### removing duplicate because engineered:


### adding duplicate lines for Achilles

In [ ]:
mergedsegments = mergedsegments[['DepMap_ID', 'Chromosome', 'Start', 'End', 'Segment_Mean', 'Num_Probes', 'Status', 'Source']].sort_values(by=['DepMap_ID', 'Chromosome', 'Start', 'End']).reset_index(drop=True)
#setting amplification status to U for X chromosome as it is artificially amplified in female samples:
mergedsegments.loc[mergedsegments[mergedsegments.Chromosome=="X"].index,'Status'] = 'U'

In [ ]:
mergedsegments = mut.manageGapsInSegments(mergedsegments, cyto=cyto)
mergedgenecn = mut.toGeneMatrix(mergedsegments, gene_mapping).apply(lambda x: np.log2(1+x))
set(mergedsegments.Source)

In [ ]:
match, corr= rna.findClosestMatching(mergedgenecn, prevgenecn, closest=True, returncorr=True)
corr.loc[set(corr.index) & set(corr.columns)][set(corr.index) & set(corr.columns)]

In [ ]:
cn.plotCNchanges(mergedgenecn, prevgenecn.apply(lambda x: np.log2(1+x)), mergedsegments, prevsegments)

In [ ]:
if(mergedgenecn.values.max() > 100):
    print("\n\n\nTOO HIGH, not LOG2 transformed!")
if(len(mergedgenecn.index) > len(set(mergedgenecn.index))):
    print("Duplicate CL, not reprioritized well!")

In [ ]:
mergedgenecn.isna().sum().sum()

In [ ]:
gene_expected_count = tc.get(name='depmap-a0ab', file='CCLE_expression_proteincoding_genes_expected_count')

In [ ]:
_, ax = plt.subplots()
rna.rnaseqcorrelation(mergedgenecn.fillna(0), gene_expected_count.fillna(0), ax, name="20Q4")
rna.rnaseqcorrelation(prevgenecn[prevgenecn.index.isin(mergedgenecn.index.tolist())], gene_expected_count.fillna(0), ax, name="20Q3")

In [ ]:
h.compareDfs(mergedgenecn, tc.get(name='depmap-a0ab', file='CCLE_gene_cn'))
#h.compareDfs(mergedsegments, tc.get(name='depmap-a0ab', file='CCLE_segment_cn'))

In [ ]:
mergedgenecn.to_csv('temp/all_'+release+'_gene_cn.csv')
mergedsegments.to_csv('temp/all_'+release+'_segment.csv', index=False)

# uploading on taiga

## CN

In [ ]:
tc.update_dataset(changes_description = "updated to new " + samplesetname + " release! (updated from relabelling see google drive file for more info)",
                dataset_permaname="cn-wes-achilles-4dcd",
                 upload_files=[
                     {
                        "path": 'temp/all_'+release+'_segment.csv',
                        "name": 'all_'+release+'_segment',
                        "format": "TableCSV",
                        "encoding": "utf-8"
                    },
                    {
                        "path": 'temp/all_'+release+'_gene_cn.csv',
                        "name": 'all_'+release+'_gene_cn',
                        "format": "NumericMatrixCSV",
                        "encoding": "utf-8"
                    },
                 ],
                  dataset_description="""
# Copy Number

Combined segment and gene-level CN calls from Broad WES, Sanger WES, and Broad SNP. Relative CN, log2(x+1) transformed.

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal. Must use subsetted dataset instead. These data will not make it on the portal starting 19Q1. With the DMC portal, there is new cell line release prioritization as to which lines can be included, so a new taiga dataset will be created containing CN for the portal.

These data are generated for Achilles to pull from to run CERES.


Gene level CN data:

__Rows__: DepMap cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Segment level data:

__Columns__: DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean""")